In [23]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [5]:
olx_df = pd.read_csv('data\df_final_v1.csv')
example_df = pd.read_csv('data\example_data_v1.csv')

In [6]:
olx_df.columns, example_df.columns

(Index(['Unnamed: 0', 'description', 'preprocessed_description', 'index',
        'tfidf_sum', 'tfidf_mean', 'emotions_sum', 'emotions_mean',
        'text_length', 'capital_letters_count', 'numbers_count',
        'question_marks_count', 'currency_signs_count', 'capital_words_count',
        'possible_email', 'possible_address', 'non_polish_char_count',
        'possible_phone_numbers', 'keywords_counter'],
       dtype='object'),
 Index(['Unnamed: 0', 'description', 'preprocessed_description', 'label',
        'index', 'tfidf_sum', 'tfidf_mean', 'emotions_sum', 'emotions_mean',
        'text_length', 'capital_letters_count', 'numbers_count',
        'question_marks_count', 'currency_signs_count', 'capital_words_count',
        'possible_email', 'possible_address', 'non_polish_char_count',
        'possible_phone_numbers', 'keywords_counter'],
       dtype='object'))

In [54]:
true_labels_indices = example_df.loc[:,'label']

In [56]:
df1 = olx_df.drop(columns=["Unnamed: 0","description", "preprocessed_description", ])
df2 = example_df.drop(columns=["Unnamed: 0","description", "preprocessed_description","label"])

df1["label"] = -1
df2["label"] = true_labels_indices

In [57]:
df1.columns, df2.columns   

(Index(['index', 'tfidf_sum', 'tfidf_mean', 'emotions_sum', 'emotions_mean',
        'text_length', 'capital_letters_count', 'numbers_count',
        'question_marks_count', 'currency_signs_count', 'capital_words_count',
        'possible_email', 'possible_address', 'non_polish_char_count',
        'possible_phone_numbers', 'keywords_counter', 'label'],
       dtype='object'),
 Index(['index', 'tfidf_sum', 'tfidf_mean', 'emotions_sum', 'emotions_mean',
        'text_length', 'capital_letters_count', 'numbers_count',
        'question_marks_count', 'currency_signs_count', 'capital_words_count',
        'possible_email', 'possible_address', 'non_polish_char_count',
        'possible_phone_numbers', 'keywords_counter', 'label'],
       dtype='object'))

In [59]:
final_df = pd.concat([df1, df2], axis=0)
final_df

,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,numbers_count,question_marks_count,currency_signs_count,capital_words_count,possible_email,possible_address,non_polish_char_count,possible_phone_numbers,keywords_counter,label
0,0,8.381325,0.000282,76.0,0.002558,1165,29,16,0,1,0,NaN,NaN,66,NaN,2,-1
1,1,8.790453,0.000296,89.0,0.002995,1342,18,22,0,0,1,NaN,NaN,47,"48 14 610 01 40, 48 14 610 01 45",1,-1
2,2,9.400964,0.000316,90.0,0.003029,1276,141,27,0,1,17,NaN,NaN,72,NaN,0,-1
3,3,9.881147,0.000333,93.0,0.003130,1361,27,0,4,0,2,NaN,NaN,34,NaN,2,-1
4,4,8.143853,0.000274,60.0,0.002019,892,20,19,0,0,0,NaN,NaN,42,NaN,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,34,6.892056,0.004268,47.0,0.029084,645,8,0,0,0,0,NaN,NaN,18,NaN,0,0
33,35,11.312366,0.007005,175.0,0.108292,1937,676,14,7,0,115,NaN,NaN,54,NaN,6,0
34,36,11.125390,0.006889,151.0,0.093441,1988,37,13,3,0,1,NaN,NaN,73,NaN,7,0
35,37,11.822965,0.007321,170.0,0.105198,2185,36,7,0,0,1,NaN,NaN,62,NaN,6,0


In [60]:
final_df['ispossible_address'] =final_df['possible_address'].isnull().astype(int)
final_df['ispossible_email'] =final_df['possible_email'].isnull().astype(int)
final_df['ispossible_phone_numbers'] =final_df['possible_phone_numbers'].isnull().astype(int)

In [61]:
final_df2 = final_df.drop(columns=["possible_address", "possible_email", "possible_phone_numbers", "label", "index"])


In [102]:
# Convert DataFrame to numpy array
data_array = final_df2.values

# Create an imputer to fill missing values with the mean of each column
imputer = SimpleImputer(strategy='mean')

# Apply imputation to fill missing values
data_imputed = imputer.fit_transform(data_array)

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

# KMeans
n_clusters = 8  # Specify the desired number of clusters
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(data_imputed)
cluster_labels = kmeans.labels_

final_df['KMeans_label'] = cluster_labels

from sklearn.cluster import AgglomerativeClustering

# Agglomerative Clustering
n_clusters = 8  # Specify the desired number of clusters
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)
cluster_labels = agg_clustering.fit_predict(data_array)

final_df['Agglomerative Clustering'] = cluster_labels


from sklearn.cluster import DBSCAN

# Perform DBSCAN clustering
# eps = 0.5  # The maximum distance between two samples to be considered in the same neighborhood
# min_samples = 5  # The minimum number of samples in a neighborhood to be considered as a core point
# dbscan = DBSCAN(eps=eps, min_samples=min_samples)
# cluster_labels = dbscan.fit_predict(data_array)

# final_df["DBSCAN"] = cluster_labels


from sklearn.mixture import GaussianMixture

# Perform GMM clustering
n_components = 8  # Specify the desired number of components/clusters
gmm = GaussianMixture(n_components=n_components)
gmm.fit(data_array)
cluster_labels = gmm.predict(data_array)

final_df["GMM"] = cluster_labels


from sklearn.cluster import MeanShift

# Perform Mean Shift clustering
# bandwidth = 0.8  # The bandwidth used in the kernel density estimation
# mean_shift = MeanShift(bandwidth=bandwidth)
# cluster_labels = mean_shift.fit_predict(data_array)

# final_df["Mean Shift"] = cluster_labels


c:\Users\wkosi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [103]:
final_df

,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,numbers_count,question_marks_count,currency_signs_count,...,keywords_counter,label,ispossible_address,ispossible_email,ispossible_phone_numbers,KMeans_label,Agglomerative Clustering,DBSCAN,GMM,Mean Shift
0,0,8.381325,0.000282,76.0,0.002558,1165,29,16,0,1,...,2,-1,1,1,1,6,7,-1,3,2459
1,1,8.790453,0.000296,89.0,0.002995,1342,18,22,0,0,...,1,-1,1,1,0,6,7,-1,4,2458
2,2,9.400964,0.000316,90.0,0.003029,1276,141,27,0,1,...,0,-1,1,1,1,6,7,-1,1,2456
3,3,9.881147,0.000333,93.0,0.003130,1361,27,0,4,0,...,2,-1,1,1,1,6,7,-1,1,2453
4,4,8.143853,0.000274,60.0,0.002019,892,20,19,0,0,...,0,-1,1,1,1,0,7,-1,3,2451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,34,6.892056,0.004268,47.0,0.029084,645,8,0,0,0,...,0,0,1,1,1,0,5,-1,6,2395
33,35,11.312366,0.007005,175.0,0.108292,1937,676,14,7,0,...,6,0,1,1,1,6,2,-1,2,2392
34,36,11.125390,0.006889,151.0,0.093441,1988,37,13,3,0,...,7,0,1,1,1,6,2,-1,2,2391
35,37,11.822965,0.007321,170.0,0.105198,2185,36,7,0,0,...,6,0,1,1,1,6,2,-1,2,2388


In [104]:
comparison = final_df[final_df["label"]!=-1].loc[:,["label", "KMeans_label","Agglomerative Clustering",  "GMM"]]

In [105]:
comparison.loc[:,["label","KMeans_label"]].value_counts()

label  KMeans_label
0      6               10
       2                8
1      6                7
0      0                5
1      0                5
0      3                1
1      2                1
Name: count, dtype: int64

In [106]:
comparison.loc[:,["label","Agglomerative Clustering"]].value_counts()

label  Agglomerative Clustering
0      6                           8
1      7                           6
0      2                           5
       5                           5
       7                           5
1      5                           5
0      3                           1
1      2                           1
       6                           1
Name: count, dtype: int64

In [94]:
comparison.loc[:,["label","DBSCAN"]].value_counts()

label  DBSCAN
0      -1        24
1      -1        13
Name: count, dtype: int64

In [95]:
comparison.loc[:,["label","GMM"]].value_counts()

label  GMM
0      0      24
1      0      11
       1       2
Name: count, dtype: int64

In [96]:
comparison.loc[:,["label","Mean Shift"]].value_counts()

label  Mean Shift
0      2386          1
       2424          1
       2428          1
       2430          1
       2432          1
1      2436          1
       2437          1
       2439          1
       2441          1
       2444          1
       2445          1
       2448          1
       2449          1
       2452          1
       2454          1
       2455          1
       2457          1
0      2426          1
       2423          1
       2388          1
       2421          1
       2391          1
       2392          1
       2395          1
       2396          1
       2399          1
       2401          1
       2403          1
       2405          1
       2406          1
       2409          1
       2410          1
       2412          1
       2414          1
       2416          1
       2419          1
1      2460          1
Name: count, dtype: int64

In [82]:
final_df2

,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,numbers_count,question_marks_count,currency_signs_count,capital_words_count,non_polish_char_count,keywords_counter,ispossible_address,ispossible_email,ispossible_phone_numbers
0,0,8.381325,0.000282,76.0,0.002558,1165,29,16,0,1,0,66,2,1,1,1
1,1,8.790453,0.000296,89.0,0.002995,1342,18,22,0,0,1,47,1,1,1,0
2,2,9.400964,0.000316,90.0,0.003029,1276,141,27,0,1,17,72,0,1,1,1
3,3,9.881147,0.000333,93.0,0.003130,1361,27,0,4,0,2,34,2,1,1,1
4,4,8.143853,0.000274,60.0,0.002019,892,20,19,0,0,0,42,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,34,6.892056,0.004268,47.0,0.029084,645,8,0,0,0,0,18,0,1,1,1
33,35,11.312366,0.007005,175.0,0.108292,1937,676,14,7,0,115,54,6,1,1,1
34,36,11.125390,0.006889,151.0,0.093441,1988,37,13,3,0,1,73,7,1,1,1
35,37,11.822965,0.007321,170.0,0.105198,2185,36,7,0,0,1,62,6,1,1,1
